# Hybrid Search using RRF

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/02-hybrid-search.ipynb)

이 예에서는 상호 순위 융합 알고리즘을 사용하여 BM25와 kNN 의미 검색 결과를 결합합니다.
[빠른 시작](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb) 가이드에서 사용한 것과 동일한 데이터세트를 사용하겠습니다.
추가 구성 없이 바로 하이브리드 검색에 RRF를 사용할 수 있습니다.

또한 RRF 순위가 기본 수준에서 작동하는 방식을 보여주는 간단한 예제에 대한 연습도 제공합니다.

# 🧰 Requirements

이 예에서는 다음이 필요합니다.

- 최소 **4GB 기계 학습 노드**


# Install packages and initialize the Elasticsearch Python client

시작하려면 Python 클라이언트를 사용하여 Elastic 배포에 연결해야 합니다.
먼저 이 예제에 필요한 패키지를 `pip` 설치해야 합니다

In [21]:
!pip install elasticsearch>=8.9.0
!pip install sentence_transformers
!pip install torch

다음으로 `elasticsearch` 모듈과 `getpass` 모듈을 가져와야 합니다.
`getpass`는 Python 표준 라이브러리의 일부이며 자격 증명을 안전하게 요청하는 데 사용됩니다.

In [22]:
import os
cwd = os.getcwd()

try :
    os.mkdir(cwd + "/models") 
except:
    pass

해당 경로가 이미 존재합니다.


In [23]:
os.chdir(cwd + "/models")

try :
    os.system("git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2")
except:
    print('이미 모델이 존재합니다.')

os.chdir(cwd)

In [24]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer(cwd + '/models/all-MiniLM-L6-v2', device=device)

이제 Python Elasticsearch 클라이언트를 인스턴스화할 수 있습니다.
먼저 사용자에게 URL, 비밀번호와 'CERT 파일 경로'를 묻는 메시지를 표시합니다.

🔐 참고: `getpass`를 사용하면 자격 증명을 터미널에 표시하거나 메모리에 저장하지 않고 사용자에게 자격 증명을 안전하게 묻는 메시지를 표시할 수 있습니다.

그런 다음 `Elasticsearch` 클래스의 인스턴스를 인스턴스화하는 `클라이언트` 개체를 만듭니다.

In [25]:
from getpass import getpass

ES_URL = input('Elasticsearch URL: ')
ES_ID = "elastic" 
ES_PW = getpass('elastic user PW: ')
CERT_PATH = input('Elasticsearch cer 파일 경로: ')

# Create the client instance
client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_ID, ES_PW),
    ca_certs=CERT_PATH
)

클라이언트가 이 테스트에 연결되었는지 확인하세요.

In [26]:
print(client.info())

{'name': 'NOTESWEETHOME', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'L4MsufEIRvWXg2zAaORhYw', 'version': {'number': '8.9.1', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': 'a813d015ef1826148d9d389bd1c0d781c6e349f0', 'build_date': '2023-08-10T05:02:32.517455352Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Create Elasticsearch index with required mappings

밀집된 벡터 저장 및 검색을 지원하려면 필드를 추가해야 합니다.
`title` 필드의 밀집 벡터 표현을 저장하는 데 사용되는 아래의 `title_Vector` 필드를 참고하세요.

In [32]:
if client.indices.exists(index="rrf_book_index"):
    client.indices.delete(index="rrf_book_index")

# Create the index
client.indices.create(
    index='rrf_book_index', 
    mappings={
        "properties": {
            "title": {"type": "text"},
            "authors": {"type": "keyword"},
            "summary": {"type": "text"},
            "publish_date": {"type": "date"},
            "num_reviews": {"type": "integer"},
            "publisher": {"type": "keyword"},
            "title_vector": { 
                "type": "dense_vector", 
                "dims": 384, 
                "index": "true", 
                "similarity": "dot_product" 
            }
        }
    })

이미 인덱스가 존재합니다


## Dataset

일부 데이터를 색인화해 보겠습니다.
문장 변환기 모델을 사용하여 `title` 필드를 삽입하고 있다는 점에 유의하세요.
색인이 생성되면 문서에 부동 소수점 값의 벡터가 포함된 `title_Vector` 필드(`"type": "dense_Vector"`)가 포함되어 있음을 알 수 있습니다.
이는 벡터 공간에 `title` 필드를 삽입하는 것입니다.
이 필드를 사용하여 kNN을 사용하여 의미 검색을 수행합니다.

In [28]:
books = [
    {
        "title": "The Pragmatic Programmer: Your Journey to Mastery",
        "authors": ["andrew hunt", "david thomas"],
        "summary": "A guide to pragmatic programming for software engineers and developers",
        "publish_date": "2019-10-29",
        "num_reviews": 30,
        "publisher": "addison-wesley"
    },
    {
        "title": "Python Crash Course",
        "authors": ["eric matthes"],
        "summary": "A fast-paced, no-nonsense guide to programming in Python",
        "publish_date": "2019-05-03",
        "num_reviews": 42,
        "publisher": "no starch press"
    },
    {
        "title": "Artificial Intelligence: A Modern Approach",
        "authors": ["stuart russell", "peter norvig"],
        "summary": "Comprehensive introduction to the theory and practice of artificial intelligence",
        "publish_date": "2020-04-06",
        "num_reviews": 39,
        "publisher": "pearson"
    },
    {
        "title": "Clean Code: A Handbook of Agile Software Craftsmanship",
        "authors": ["robert c. martin"],
        "summary": "A guide to writing code that is easy to read, understand and maintain",
        "publish_date": "2008-08-11",
        "num_reviews": 55,
        "publisher": "prentice hall"
    },
    {
        "title": "You Don't Know JS: Up & Going",
        "authors": ["kyle simpson"],
        "summary": "Introduction to JavaScript and programming as a whole",
        "publish_date": "2015-03-27",
        "num_reviews": 36,
        "publisher": "oreilly"
    },
    {
        "title": "Eloquent JavaScript",
        "authors": ["marijn haverbeke"],
        "summary": "A modern introduction to programming",
        "publish_date": "2018-12-04",
        "num_reviews": 38,
        "publisher": "no starch press"
    },
    {
        "title": "Design Patterns: Elements of Reusable Object-Oriented Software",
        "authors": ["erich gamma", "richard helm", "ralph johnson", "john vlissides"],
        "summary": "Guide to design patterns that can be used in any object-oriented language",
        "publish_date": "1994-10-31",
        "num_reviews": 45,
        "publisher": "addison-wesley"
    },
    {
        "title": "The Clean Coder: A Code of Conduct for Professional Programmers",
        "authors": ["robert c. martin"],
        "summary": "A guide to professional conduct in the field of software engineering",
        "publish_date": "2011-05-13",
        "num_reviews": 20,
        "publisher": "prentice hall"
    },
    {
        "title": "JavaScript: The Good Parts",
        "authors": ["douglas crockford"],
        "summary": "A deep dive into the parts of JavaScript that are essential to writing maintainable code",
        "publish_date": "2008-05-15",
        "num_reviews": 51,
        "publisher": "oreilly"
    },
    {
        "title": "Introduction to the Theory of Computation",
        "authors": ["michael sipser"],
        "summary": "Introduction to the theory of computation and complexity theory",
        "publish_date": "2012-06-27",
        "num_reviews": 33,
        "publisher": "cengage learning"
    },
]

## Index documents

우리의 데이터 세트는 영화 제목과 설명 사전이 포함된 Python 목록입니다.
문서를 일괄적으로 색인화하기 위해 `helpers.bulk` 메소드를 사용할 것입니다.

다음 코드는 책 목록을 반복하고 수행할 작업 목록을 만듭니다.
각 작업은 Elasticsearch 인덱스에 대한 "인덱스" 작업을 포함하는 사전입니다.
책 제목은 선택한 모델을 사용하여 인코딩되고, 인코딩된 벡터는 책 문서에 추가됩니다.
그러면 책 문서가 작업 목록에 추가됩니다.

마지막으로 인덱스 이름과 작업 목록을 지정하는 `bulk` 메서드를 호출합니다.

In [29]:
actions = []
for book in books:
    actions.append({"index": {"_index": "rrf_book_index"}})
    titleEmbedding = model.encode(book["title"]).tolist()
    book["title_vector"] = titleEmbedding
    actions.append(book)

client.bulk(index="rrf_book_index", operations=actions)

ObjectApiResponse({'took': 79, 'errors': False, 'items': [{'index': {'_index': 'rrf_book_index', '_id': 'WNWIZIoBKWsdc971WidI', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 10, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'WdWIZIoBKWsdc971WidI', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 11, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'WtWIZIoBKWsdc971WidI', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 12, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'W9WIZIoBKWsdc971WidI', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 13, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'XNWIZIoBKWsdc971WidI', '_version': 1, 'resu

## Pretty printing Elasticsearch responses

Elasticsearch 응답을 읽을 수 있는 형식으로 출력하는 helper function입니다.

In [30]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        rank = hit['_rank']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nRank: {rank}")
        print(pretty_output)

# Querying Documents with Hybrid Search

이제 두 가지 다른 검색 전략을 사용하여 쿼리를 수행해야 합니다.
- "all-MiniLM-L6-v2" 임베딩 모델을 이용한 의미 검색
- "제목" 필드를 이용한 키워드 검색

그런 다음 [RRF(Reciprocal Rank Fusion)](https://www.elastic.co/guide/en/elasticsearch/reference/current/rrf.html)를 사용하여 점수의 균형을 맞추고 순위가 매겨진 문서의 최종 목록을 제공합니다. 관련순서. RRF는 다양한 정보 검색 전략의 결과를 결합하기 위한 순위 알고리즘입니다.

**참고** _score는 null이며 대신 _rank를 사용하여 최상위 문서를 표시합니다.

In [31]:
response = client.search(
    index="rrf_book_index", 
    size=5, 
    query={
        "match": {
            "summary": "python programming"
        }
    }, 
    knn={
        "field": "title_vector",
        "query_vector" : model.encode("python programming").tolist(), # generate embedding for query so it can be compared to `title_vector`
        "k": 5,
        "num_candidates": 10
    },
    rank={
        "rrf": {}
    }
)

pretty_response(response)


ID: T9ViZIoBKWsdc971ECdt
Publication date: 2019-05-03
Title: Python Crash Course
Summary: A fast-paced, no-nonsense guide to programming in Python
Rank: 1

ID: TtViZIoBKWsdc971ECds
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Rank: 2

ID: U9ViZIoBKWsdc971ECdt
Publication date: 2018-12-04
Title: Eloquent JavaScript
Summary: A modern introduction to programming
Rank: 3

ID: UtViZIoBKWsdc971ECdt
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Rank: 4

ID: V9ViZIoBKWsdc971ECdt
Publication date: 2012-06-27
Title: Introduction to the Theory of Computation
Summary: Introduction to the theory of computation and complexity theory
Rank: 5
